# Task 1 - Quantum Computing

Note, at time of writing task 1 (06/03/2023), there is a bug in the pennylane library which produces an AttributeError when calling the qml.RX,qml.RY functions, detailed at: 

https://github.com/PennyLaneAI/pennylane/issues/3867

Hence, in test-env using autoray 0.6.0

In [2]:
import pennylane as qml
from pennylane import numpy as np

In [35]:
circuit1Device = qml.device('default.qubit', wires=5)

def wireWiseHadamard(wires: list):
    '''Performs hadamard operations on each qubit in the list of wires'''
    for wire in wires:
        qml.Hadamard(wires=wire)
        
def pairWiseCNOT(device):
    '''performs CNOT operations on pairwise wires in the device for ascending labels, i.e for N 
    wires, performs CNOT on (0,1), (1,2), ... , (N-1, N)'''
    
    wires = list(device.wires)
    
    for wire in range(len(wires)-1):
        qml.CNOT(wires=[wire, wire+1])
        

        
@qml.qnode(task1Device)
def circuit1(xAxisRotationAngle: float):
    
    wireWiseHadamard(list(circuit1Device.wires))
    pairWiseCNOT(circuit1Device)
    qml.SWAP(wires=[0,4])
    qml.RX(xAxisRotationAngle, wires=2) #arbitrarily chosen 3rd qubit
    return qml.expval(qml.PauliZ(wires=2))

    
circuit1Drawer = qml.draw(circuit1)
print(circuit1Drawer(np.pi/2))

    
    
    

0: ──H─╭●──────────╭SWAP───────────┤     
1: ──H─╰X─╭●───────│───────────────┤     
2: ──H────╰X─╭●────│──────RX(1.57)─┤  <Z>
3: ──H───────╰X─╭●─│───────────────┤     
4: ──H──────────╰X─╰SWAP───────────┤     


## Task 1 - Circuit 2 

This task requires 6 qubits as we require two ancilla qubits (since we are performing a swap test on two pairs of qubits). 

In [43]:
circuit2Device = qml.device('default.qubit', wires=6)

def swapTest(wires: list, ancilla: int):
    qml.Hadamard(wires = ancilla)
    
    cswapWires = [ancilla] + wires
    qml.CSWAP(wires = cswapWires)
    qml.Hadamard(wires=ancilla)
    
    

@qml.qnode(circuit2Device)
def circuit2():
    
    qml.Hadamard(wires=0)
    qml.RX(np.pi/3, wires = 1)
    wireWiseHadamard([2,3])
    swapTestq1q2 = swapTest([0,1], 4)
    swapTestq3q4 = swapTest([2,3], 5)
    return qml.expval(qml.PauliZ(wires=4)), qml.expval(qml.PauliZ(wires=5))


circuit2Drawer = qml.draw(circuit2)
print(circuit2Drawer())
circuit2()
#shows that after the prior operations in the circuit, the states |q1>,|q2> 
#are orthogonal and |q3>, |q4> are colinear (i.e multiples of eachother)

0: ──H────────╭SWAP─────────────┤     
1: ──RX(1.05)─├SWAP─────────────┤     
2: ──H────────│────────╭SWAP────┤     
3: ──H────────│────────├SWAP────┤     
4: ──H────────╰●─────H─│────────┤  <Z>
5: ──H─────────────────╰●─────H─┤  <Z>


tensor([0.5, 1. ], requires_grad=True)